In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import re

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MaxNLocator

%matplotlib inline

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
#for adding heavy chain
complementator = str.maketrans("ACGT", "TGCA") 

In [4]:
df_category_cancers = pd.read_csv('../data/processed/df_category_cancers_pqs.csv')

In [14]:
g4pqsconf = pd.read_csv('../data/processed/g4pqsconf.csv')

In [ ]:
df_cancers = 

In [5]:
df_category_cancers

,G4_id,pos_in_seq,pos_in_ref,nuc,category,A>C,A>G,A>T,C>A,C>G,C>T,G>A,G>C,G>T,T>A,T>C,T>G,number_of_Mut
0,1,0,295,C,Tetrad,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,1,296,C,Tetrad,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
2,1,2,297,A,Bulges,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,1,3,298,C,Tetrad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,4,299,C,Tetrad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2418,67,23,16376,C,Tetrad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2419,67,24,16377,C,Tetrad,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2420,67,25,16378,C,Tetrad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2421,67,26,16379,C,Tetrad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df_category_cancers[df_category_cancers['category'] == 'Loop'] \
    .groupby(['G4_id']) \
    .agg({'A>C':'sum','A>G':'sum','A>T':'sum',
          'C>A':'sum','C>G':'sum','C>T':'sum',
          'G>A':'sum','G>C':'sum','G>T':'sum',
          'T>A':'sum','T>C':'sum','T>G':'sum'}) \
    .reset_index()

,G4_id,A>C,A>G,A>T,C>A,C>G,C>T,G>A,G>C,G>T,T>A,T>C,T>G
0,1,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,4.0,0.0
3,4,0.0,10.0,0.0,0.0,0.0,1.0,17.0,0.0,2.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,63,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0
63,64,4.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,5.0,0.0
64,65,1.0,3.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,10.0,0.0
65,66,0.0,12.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0


In [37]:
def add_columns_15(df:pd.DataFrame):
    df['15_up'] = pd.concat([df[df['bp_untill_previous_G4'] < 15][['bp_untill_previous_G4', 'start']]\
                                      .diff(axis=1)['bp_untill_previous_G4'],\
                                     df[df['bp_untill_previous_G4'] >= 15]['start']-15])
        
    df['15_down'] = pd.concat([df[df['bp_untill_next_G4'] < 15][['end', 'bp_untill_next_G4']]\
                                        .diff(axis=1)['end'], \
                                   df[df['bp_untill_next_G4'] >= 15]['end']+15])
    
    df['30_up'] = pd.concat([df[df['bp_untill_previous_G4'] < 30][['bp_untill_previous_G4', 'start']]\
                                      .diff(axis=1)['bp_untill_previous_G4'],\
                                     df[df['bp_untill_previous_G4'] >= 30]['start']-30])
        
    df['30_down'] = pd.concat([df[df['bp_untill_next_G4'] < 30][['end', 'bp_untill_next_G4']]\
                                        .diff(axis=1)['end'], \
                                   df[df['bp_untill_next_G4'] >= 30]['end']+30])
    
    df['45_up'] = pd.concat([df[df['bp_untill_previous_G4'] < 45][['bp_untill_previous_G4', 'start']]\
                                      .diff(axis=1)['bp_untill_previous_G4'],\
                                     df[df['bp_untill_previous_G4'] >= 45]['start']-45])
        
    df['45_down'] = pd.concat([df[df['bp_untill_next_G4'] < 45][['end', 'bp_untill_next_G4']]\
                                        .diff(axis=1)['end'], \
                                   df[df['bp_untill_next_G4'] >= 45]['end']+45])
    return df

In [40]:
g4pqsconf = add_columns_15(g4pqsconf).fillna(0)